In [1]:
load('/NumericalSemigroup.sage')
load('/KunzPoset.sage')

true
Successfully loaded Singular


In [3]:
#gives a list of facet intersections which should be ridges according to the rule 
#the rule being that wlog i+j neq k and its not a quadribigeminal face, or it is a tripling face 
def rulechecker(m):
    ruleridges = []
    rulenonridges = []
    for i in [1..m-1]:
        for j in [1..m-1]:
            if i+j == m:
                continue
            if i > j:
                continue
            for k in [1..m-1]:
                for l in [1..m-1]:
                    if k+l == m:
                        continue
                    if k > l:
                        continue
                    if k < i:
                        continue
                    if  k==i and l < j:
                        continue
                    Facet1 = [0]*(m)
                    Facet2 = [0]*(m)   
                    Facet1[i] = Facet1[i]+1
                    Facet1[j] = Facet1[j]+1
                    Facet1[(i+j)%m] = Facet1[(i+j)%m]-1
                    Facet2[k] = Facet2[k]+1
                    Facet2[l] = Facet2[l]+1
                    Facet2[(k+l)%m] = Facet2[(k+l)%m]-1
                    #thus far we have built the facet equalities for the facets we will be intersecting
                    if Facet1 == Facet2: #don't want them to be the same
                        continue
                    #these next few loops will count the tripling ridges
                    if i == j:
                        if k == i:
                            if l == (2*i)%m:
                                ruleridges.append(((i,j),(k,l)))
                                continue
                        if l == i:
                            if k == (2*i)%m:
                                ruleridges.append(((i,j),(k,l)))
                                continue
                    if k == l:
                        if i == k:
                            if j == (2*k)%m:
                                ruleridges.append(((i,j),(k,l))) 
                                continue
                        if j == k:
                            if i == (2*k)%m:
                                ruleridges.append(((i,j),(k,l)))
                                continue
                    #these next few loops will remove the non-esker faces
                    if i == (k+l)%m:
                        rulenonridges.append(((i,j),(k,l)))
                        continue
                    if j == (k+l)%m:
                        rulenonridges.append(((i,j),(k,l)))
                        continue
                    if k == (i+j)%m:
                        rulenonridges.append(((i,j),(k,l)))
                        continue
                    if l == (i+j)%m:
                        rulenonridges.append(((i,j),(k,l)))
                        continue
                    #this next bit removes quadribigeminal faces
                    if m%4 == 0:
                        n = m/4
                        if j == i+2*n:
                            if k == i+n:
                                if l == i+3*n:
                                    rulenonridges.append(((i,j),(k,l)))
                                    continue
                    ruleridges.append(((i,j),(k,l)))
    return ruleridges

In [4]:
#gives a list of facet intersections that are ridges, taken from the actual polyhedron
def conechecker(m):
    cone = Polyhedron(ieqs = KunzPoset.KunzInequalities(m))
    cone.f_vector()
    actualridges = []
    actualnonridges = []
    for ieq1 in KunzPoset.KunzInequalities(m):
        for jeq2 in KunzPoset.KunzInequalities(m):
            if ieq1 >= jeq2:
                continue
            testridge = cone.intersection(Polyhedron(eqns = [ieq1,jeq2]))
            #pulling information out of the equations to have the ridges in the same format as above
            for i in [1..m-1]:
                if ieq1[i] == 2:
                    a = i
                    b = i
                    break
                elif ieq1[i] == 1:
                    a = i
                    for i2 in [i+1..m-1]:
                        if ieq1[i2] ==1:
                            b = i2
                            break
                    break
            for j in [1..m-1]:
                if jeq2[j] == 2:
                    c = j
                    d = j
                    break
                elif jeq2[j] == 1:
                    c = j
                    for j2 in [j+1..m-1]:
                        if jeq2[j2] == 1:
                            d = j2
                            break
                    break
            #checking dimensions
            #ensuring the ((,),(,)) representation has the same ordering as above
            if testridge.dim() == m-3:
                if a < c:
                    actualridges.append(((a,b),(c,d))) 
                if a == c and b <= d:
                    actualridges.append(((a,b),(c,d)))
                if a == c and b > d:
                    actualridges.append(((c,d),(a,b)))
                if a > c:
                    actualridges.append(((c,d),(a,b)))
            else:
                if a < c:
                    actualnonridges.append(((a,b),(c,d)))
                if a == c and b <= d:
                    actualnonridges.append(((a,b),(c,d)))
                if a == c and b > d:
                    actualnonridges.append(((c,d),(a,b)))           
                if a > c:
                    actualnonridges.append(((c,d),(a,b)))
    return actualridges

In [5]:
#compare the two lists above
def letscompare(m):
    ruleridges = rulechecker(m)
    actualridges = conechecker(m)
    print('Not counted as a ridge by rule, but is a ridge:')
    print([ridge for ridge in actualridges if ridge not in ruleridges])
    print('Counted as a ridge by rule, but is not a ridge:')
    print([ridge for ridge in ruleridges if ridge not in actualridges])

In [6]:
#test it! we want both lists to be empty
#recall that this rule works for m >= 5
letscompare(8)

Not counted as a ridge by rule, but is a ridge:
[]
Counted as a ridge by rule, but is not a ridge:
[]


In [15]:
#count the facets
#for m >=3
def facetcounter(m):
    Cm = Polyhedron(ieqs = KunzPoset.KunzInequalities(m))
    nfacets = Cm.f_vector()[m-1]
    if m%2 == 0:
        cfacets = (m*(m-2))/2
    if m%2 != 0:
        cfacets = ((m-1)^2)/2
    #print(nfacets)
    #print(cfacets)
    if nfacets == cfacets:
        print('nice')
        print(cfacets)
        print(nfacets)
    if nfacets != cfacets:
        print('uh oh')
        print(cfacets)
        print(nfacets)

In [16]:
#check it!
facetcounter(5)

nice
8
8


In [9]:
#count the ridges
#again, recall this formula works for m >=5
def ridgecounter(m):
    Cm = Polyhedron(ieqs = KunzPoset.KunzInequalities(m))
    nfacets = Cm.f_vector()[m-1] #actual number of facets
    nridges = Cm.f_vector()[m-2] #actual number of ridges
    dtrip = 0 #number of tripling faces
    dquad = 0 #number of quadribigeminal faces
    if m%6 == 0:
        dtrip = m-4
    if m%6 == 1:
        dtrip = m-1
    if m%6 == 2:
        dtrip = m-2
    if m%6 == 3:
        dtrip = m-3
    if m%6 == 4:
        dtrip = m-2
    if m%6 == 5:
        dtrip = m-1
    if m%4 == 0:
        dquad = m/4-1
    countedridges = binomial(nfacets,2) - nfacets*(m-2) + (m-1)*(m-2)/2 + dtrip - dquad
    if countedridges == nridges:
        print('nice')
    if countedridges != nridges:
        print('dang')
    print(nridges)
    print(countedridges)
    

In [10]:
ridgecounter(7)

nice
84
84


In [17]:
#just in case you really like a quasipolynomial fully written out
def ridgecounter2(m):
    Cm = Polyhedron(ieqs = KunzPoset.KunzInequalities(m))
    nridges = Cm.f_vector()[m-2]
    if m%12 == 0:
        countedridges = m^4/8-m^3+11*m^2/4 - 9*m/4-2
    if m%12 == 1:
        countedridges = m^4/8 - m^3 + 3*m^2 - 3*m + 7/8
    if m%12 == 2:
        countedridges = m^4/8-m^3+11*m^2/4-2*m-1
    if m%12 == 3:
        countedridges = m^4/8 - m^3 + 3*m^2 - 3*m -9/8
    if m%12 == 4:
        countedridges = m^4/8-m^3+11*m^2/4 - 9*m/4
    if m%12 == 5:
        countedridges = m^4/8 - m^3 + 3*m^2 - 3*m + 7/8
    if m%12 == 6:
        countedridges = m^4/8-m^3+11*m^2/4-2*m-3
    if m%12 == 7:
        countedridges = m^4/8 - m^3 + 3*m^2 - 3*m + 7/8
    if m%12 == 8:
        countedridges = m^4/8-m^3+11*m^2/4 - 9*m/4
    if m%12 == 9:
        countedridges = m^4/8 - m^3 + 3*m^2 - 3*m -9/8
    if m%12 == 10:
        countedridges = m^4/8-m^3+11*m^2/4-2*m-1
    if m%12 == 11:
        countedridges = m^4/8 - m^3 + 3*m^2 - 3*m + 7/8
    if countedridges == nridges:
        print('nice')
    if countedridges != nridges:
        print('dang')
    print(nridges)
    print(countedridges)

In [18]:
#test it!
ridgecounter2(7)

nice
84
84
